# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [14]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
# Import API key
from api_keys import geoapify_key

In [15]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,cidreira,-30.1811,-50.2056,67.06,96,100,9.06,BR,overcast clouds
1,1,tinajo,29.0788,-13.7081,70.68,78,20,5.75,ES,few clouds
2,2,kolonia town,6.9640,158.2060,86.00,74,75,5.75,FM,light rain
3,3,hamilton,39.1834,-84.5333,46.06,81,75,9.22,US,broken clouds
4,4,edinburgh of the seven seas,-37.0676,-12.3116,55.06,90,92,29.97,SH,light rain


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

# Configure the map plot
humidity = city_data_df["Humidity"].astype(int)
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    color="City",
    alpha=.75)


# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [22]:
# Narrow down cities that fit criteria and drop any results with null values
#A max temperature lower than 75 degrees but higher than 50
city_data_df = city_data_df.loc[(city_data_df["Max Temp"] < 75) & (city_data_df["Max Temp"] > 50)]

#Wind speed less than 5 m/s
city_data_df = city_data_df.loc[(city_data_df["Wind Speed"] < 5)]

#Zero cloudiness
city_data_df = city_data_df.loc[(city_data_df["Cloudiness"] == 0)]
# Drop any rows with null values
city_data_df = city_data_df.dropna()
# Display sample data
city_data_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
28,28,chios,38.3678,26.1358,60.66,94,0,2.30,GR,clear sky
66,66,altamont,42.2068,-121.7372,58.84,27,0,1.99,US,clear sky
129,129,arsin,40.9496,39.9276,62.13,82,0,3.44,TR,clear sky
173,173,meridian,43.6121,-116.3915,51.51,41,0,3.44,US,clear sky
220,220,didymoteicho,41.3481,26.4961,62.26,73,0,1.43,GR,clear sky
432,432,homer,59.6425,-151.5483,50.77,73,0,1.99,US,clear sky
503,503,huangmei,30.1924,116.0250,64.49,67,0,1.86,CN,clear sky
525,525,argo,37.6333,22.7333,65.95,73,0,4.41,GR,clear sky


### Step 3: Create a new DataFrame called `hotel_df`.

In [21]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country','Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
28,chios,GR,38.3678,26.1358,94,
66,altamont,US,42.2068,-121.7372,27,
129,arsin,TR,40.9496,39.9276,82,
173,meridian,US,43.6121,-116.3915,41,
220,didymoteicho,GR,41.3481,26.4961,73,
432,homer,US,59.6425,-151.5483,73,
503,huangmei,CN,30.1924,116.0250,67,
525,argo,GR,37.6333,22.7333,73,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [23]:
# Set parameters to search for a hotel
categories = "accommodation.hotel"
radius = 10000
limit = 20

params = {
    "categories":categories,
    "limit":limit,
    "apiKey":geoapify_key 
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address =requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
chios - nearest hotel: Aegean Sea
altamont - nearest hotel: Quality Inn
arsin - nearest hotel: Arsen Hotel
meridian - nearest hotel: Best Western
didymoteicho - nearest hotel: No hotel found
homer - nearest hotel: Driftwood Inn & Homer Seaside Lodges
huangmei - nearest hotel: No hotel found
argo - nearest hotel: Apollon Hotel


,City,Country,Lat,Lng,Humidity,Hotel Name
28,chios,GR,38.3678,26.1358,94,Aegean Sea
66,altamont,US,42.2068,-121.7372,27,Quality Inn
129,arsin,TR,40.9496,39.9276,82,Arsen Hotel
173,meridian,US,43.6121,-116.3915,41,Best Western
220,didymoteicho,GR,41.3481,26.4961,73,No hotel found
432,homer,US,59.6425,-151.5483,73,Driftwood Inn & Homer Seaside Lodges
503,huangmei,CN,30.1924,116.0250,67,No hotel found
525,argo,GR,37.6333,22.7333,73,Apollon Hotel


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [24]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng','Lat', 
    geo=True, 
    tiles= 'OSM',
    frame_width= 700,
    frame_height= 500,
    color= 'City',
    hover_cols=['Hotel Name','Country']
)


# Display the map
map_plot_2

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Hotel Name,Country)